In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

base_dir = 'c:/Users/deepa/Documents/Deeplearning CA/deep-learning-techniques-comparison-rnn-tcn'
preprocessed_dir = os.path.join(base_dir, 'preprocessed_data')

In [ ]:
with open(os.path.join(preprocessed_dir, 'har.pkl'), 'rb') as f:
    har_data = pickle.load(f)

X_train_seq = har_data['X_train_seq'].astype(np.float32)
X_val_seq = har_data['X_val_seq'].astype(np.float32)
X_test_seq = har_data['X_test_seq'].astype(np.float32)

y_train_seq = har_data['y_train_seq'].astype(np.int32)
y_val_seq = har_data['y_val_seq'].astype(np.int32)
y_test_seq = har_data['y_test_seq'].astype(np.int32)

print(f"X_train shape: {X_train_seq.shape}")
print(f"X_val shape: {X_val_seq.shape}")
print(f"X_test shape: {X_test_seq.shape}")
print(f"y_train unique classes: {np.unique(y_train_seq)}")
print(f"Sequence length: {X_train_seq.shape[1]}, Features: {X_train_seq.shape[2]}, Classes: {len(np.unique(y_train_seq))}")

In [ ]:
seq_length = X_train_seq.shape[1]
num_features = X_train_seq.shape[2]
num_classes = len(np.unique(y_train_seq))

hidden_units = 128
epochs = 50
batch_size = 32
learning_rate = 0.001

model = Sequential([
    SimpleRNN(hidden_units, activation='relu', input_shape=(seq_length, num_features), return_sequences=False),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

In [ ]:
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)

In [ ]:
y_pred = model.predict(X_test_seq)
y_pred_classes = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test_seq, y_pred_classes)
f1 = f1_score(y_test_seq, y_pred_classes, average='weighted')

print(f"\n{'='*50}")
print(f"TensorFlow Vanilla RNN - HAR Classification Results")
print(f"{'='*50}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score (weighted): {f1:.4f}")
print(f"\nConfusion Matrix:\n{confusion_matrix(y_test_seq, y_pred_classes)}")
print(f"\nClassification Report:\n{classification_report(y_test_seq, y_pred_classes)}")

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()